## Exercise 11
### Useful Functions and Methods

**1. During your research in ion channels you stumble upon the SwissProt entry (CIQ3_HUMAN) in appendix 1. You notice in the feature table (FT) that several variants and mutations of this gene exists. You want to take a closer look at this and decide that the first step is to extract the original amino acid sequence and all variations (full sequence with the appropiate amino acid changed) hereoff and put the result in a fasta file. Do not consider combinations. Since you probably are going to do this on a lot of SwissProt entries, you decide to make a program in python.** 

    a) Describe which keywords/patterns you will be looking for when parsing the file searching for the variants/mutations and the sequence.
    b) Describe a method, perhaps using pseudo code, to extract the sequence and variations/mutants and print all the different full sequences in fasta. 
    c) Implement your method in python.
    d) What kind of error checking could/should you include in your program ? Here you should name every check, which is relevant to the task, not every check it is possible to make.
    e) In what way could you generalize or extend the program ?

In [70]:
infile = open("/Users/laurasansc/github/learning_julia/files/appendix1.txt", "r")

seq = ""
flag = false # Define a variable for the sequence
mutation = Dict()
id_prot = ""

for line in eachline(infile)
    if occursin("ID", line)
        id_line = split(line)
        id_prot = id_line[2]

    elseif occursin("FT", line)
        ft_line = split(line)
        if ft_line[2] == "VARIANT" || ft_line[2] == "MUTAGEN"
            result_match = match(r"^FT\s+(\w+)\s+\d+\s+(\d+)\s+(\w+)\s*?[-][>]\s*?(\w+)", line)
            
            if ft_line[3] == ft_line[4]
                mutation[ft_line[3]] = (result_match[3], result_match[4])
            end
        end
    
    elseif occursin("//", line)
            flag = false
    elseif flag == true
            line = strip(line)
            line = replace(line, " " => "")
            seq = string(seq, line)
    elseif occursin("SQ", line)
            flag = true
    end 
end

println("Printing the results in an output file (aa_seq.txt)...")

outfile = open("aa_seq.txt", "w")

write(outfile,">",id_prot,"\tWILD TYPE\n")

for i in 1:60:length(seq)
    if (i+60) > length(seq)
        write(outfile,seq[i:length(seq)], "\n")
    else
        write(outfile, seq[i:i+60], "\n")
    end
end


for (key, value) in mutation
    value = collect(value)
    k = parse.(Int64, key)

    write(outfile, ">",id_prot," MUTATION position: ", key, " (", value[1]," => ", value[2], ")\n")
    mut_sequence = collect(seq)
    splice!(mut_sequence, k, value[1])
    mut_sequence = join(mut_sequence, "")
    for i in 1:60:length(mut_sequence)
        if (i+60) > length(seq)
            write(outfile,mut_sequence[i:length(seq)], "\n")
        else
            write(outfile, mut_sequence[i:i+60], "\n")
        end
    end
end
close(infile)
close(outfile)

Printing the results in an output file (aa_seq.txt)...


**2. You have earlier in your career made a splendid program, that calculate various scores based on amino acid sequence features. The output from your program can be seen in appendix 3, and consist of an accession number followed by 6 numbers between 0 and 1 per line (tab separated). You want to find the accession numbers with the highest and lowest average scores (average of the 6 numbers). However, you want to exclude any genes on your negative list from your calculations. These genes are listed as SwissProt IDs in appendix 4. Since GenBank accession numbers and SwissProt IDs are not identical, you need to translate between them in order to solve your problem. Fortunately you have a file, that does just that, see appendix 5, where the first item on the line is a SwissProt ID, second item is irrelevant, and third is the corresponding GenBank accession number.**

    a) Describe a method, perhaps using pseudo code, to find the data.
    b) Implement your method in python.
    c) Have you made any assumptions about the data in your algoritm ? Which ? Why ? Are they reasonable assumptions (explain) ? Could/should you do away with them (by changing the code) ?
    d) Usually, when you have this kind of problem, you want the highest 10 and lowest 10 average scores, not just the top and bottom average score. How would you solve this problem ? Will it change any assumptions i c) ?

In [116]:
infile = open("/Users/laurasansc/github/learning_julia/files/appendix3.txt", "r")
translation_ID = open("/Users/laurasansc/github/learning_julia/files/appendix5.txt", "r")
negative = open("/Users/laurasansc/github/learning_julia/files/appendix4.txt", "r")

# get average of app3
# exclude from app3 the ones in app4
# use app5 to translate from 1 id to the other

#exclusion list
exclusion_list = [line for line in eachline(negative)]

# dict to translate from SwissProtID to GenBankID and viceversa
translate = Dict()
for line in eachline(translation_ID)
	entry = split(line)
	translate[entry[3]] = entry[1]
end


# dict of the scores we want to calculate (raw data, without exclusions)
scores = Dict()
for line in eachline(infile)
    entry = split(line)
    scores[entry[1]] = [] 
    for i in 2:length(entry)
        push!(scores[entry[1]], parse.(Float64, entry[i]))
    end
end

# calculate average make dict with genID and swisprotID and then exclude negatives
average = Dict()
for (k,v) in scores
    if haskey(translate, k) == true
        if (translate[k] in exclusion_list) == false     
            average[(k,translate[k])] = sum(v)/length(v)
        end
    end
end

#open outfile
outfile = open("average_scores.txt", "w")


# sort and get top 3 and the bottom 3
sorted_scores = sort(collect(values(average)),rev=true)

# X = how many top/bottom scores to show
x = 3
# get top and bottom scores
top_scores = sorted_scores[1:x]
bottom_scores = sorted_scores[end-x:end]

#write outfile
write(outfile, "TOP SCORES \n")
for i in 1:length(top_scores)
    for (k,v) in average
        key = collect(k)
        if v == top_scores[i]
            v = round(v,digits = 4)
            line = string(key[1],"\t",key[2],"\t",v,"\n")
            write(outfile, line)
        end
    end
end


write(outfile, "BOTTOM SCORES \n")
for i in 1:length(bottom_scores)
    for (k,v) in average
        key = collect(k)
        if v == bottom_scores[i]
            v = round(v,digits = 4)
            line = string(key[1],"\t",key[2],"\t",v,"\n")
            write(outfile, line)
        end
    end
end


close(translation_ID)
close(negative)
close(infile)
close(outfile)